# Read the data

data can be dowloaded from http://www.manythings.org/anki/

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import zipfile
import io

In [ ]:
data=''
with open('../input/englishtohindi/hin.txt','rb') as readfile:
    for line in io.TextIOWrapper(readfile,'utf-8'):
        data+=line

In [ ]:
#data

In [ ]:
data=data.split('\n')
data[1:20]

In [ ]:
#data[0].split('\t')

Review the data

In [ ]:
len(data)

# Seperate the encoder and decoder input

In [ ]:
encoder_text=[]
decoder_text=[]


In [ ]:
for line in data:
    try:
        in_text,out_txt,_=line.split('\t')
        encoder_text.append(in_text)
        decoder_text.append('<start>'+out_txt+'<end>')
    except:
        pass
            

In [ ]:
encoder_text[1:10]

In [ ]:
decoder_text[1:10]

# Build input and output sequence

In [ ]:
import keras

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
encoder_t=Tokenizer()

In [ ]:
encoder_t.fit_on_texts(encoder_text)

In [ ]:
encoder_seq=encoder_t.texts_to_sequences(encoder_text)

In [ ]:
encoder_seq[1:10]

In [ ]:
encoder_max_lenth=max([len(txt) for txt in encoder_seq])

In [ ]:
encoder_max_lenth

In [ ]:
encoder_vocab_size=len(encoder_t.word_index)
encoder_vocab_size

**Same steps for decoder also**

In [ ]:
decoder_t=Tokenizer()

In [ ]:
decoder_t.fit_on_texts(decoder_text)

In [ ]:
#decoder_t.word_index.ix[1:10]
import itertools

#(itertools.islice( (decoder_t.word_index()),10)
#take(10, decoder_t.word_index())

In [ ]:
print(decoder_t.word_index.items())

In [ ]:
decoder_seq=decoder_t.texts_to_sequences(decoder_text)

In [ ]:
decoder_seq[1:10]

In [ ]:
decoder_max_length=max([len(text) for text in decoder_seq])

In [ ]:
decoder_max_length

In [ ]:
decoder_vocab_size=len(decoder_t.word_index)
decoder_vocab_size

# Padding both the sequences

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
encoder_input_data=pad_sequences(encoder_seq,maxlen=encoder_max_lenth,padding='post')
decoder_input_data=pad_sequences(decoder_seq,maxlen=decoder_max_length,padding='post')

In [ ]:
encoder_input_data.shape

In [ ]:
decoder_input_data.shape

In [ ]:
encoder_input_data[1:10]

In [ ]:
decoder_input_data[1:10]

# Building Decoder output

In [ ]:
decoder_target_data=np.zeros((decoder_input_data.shape[0],decoder_input_data.shape[1]))

In [ ]:
decoder_target_data.shape

In [ ]:
#shifting the data by one unit for each row , since it is a prediction of the next word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1]=decoder_input_data[i][j]
        


In [ ]:
max(map(max, decoder_target_data))

# Maximum value of decoder target data

In [ ]:
decoder_target_data[:26]

In [ ]:
decoder_input_data.shape

In [ ]:
decoder_target_data.shape

In [ ]:
decoder_input_data[0]

In [ ]:
decoder_target_data[0]

In [ ]:
(decoder_vocab_size)

**convert this data to one hot encoding for softmax**

In [ ]:
from keras.utils import to_categorical

In [ ]:
decoder_target_onehot=np.zeros((decoder_target_data.shape[0],decoder_target_data.shape[1],decoder_vocab_size+1))

In [ ]:
decoder_target_onehot.shape

In [ ]:
decoder_target_data.shape

In [ ]:
decoder_t.word_index

In [ ]:
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_onehot[i][j]=to_categorical(decoder_target_data[i][j],num_classes=decoder_vocab_size+1)

In [ ]:
decoder_vocab_size-1

In [ ]:
decoder_target_onehot.shape

In [ ]:
decoder_target_onehot[0]

# Building the training Model
**We are going to use the Keras functional API this time**

In [ ]:
from keras.layers import Input,Embedding,Dense,LSTM

In [ ]:
# define the embeding vector length

encoder_embeding_size=50
decoder_embeding_size=50

**Build the encoder**

In [ ]:
encoder_input=Input(shape=(encoder_max_lenth,))

In [ ]:
encoder_embedding=Embedding(encoder_vocab_size+1,encoder_embeding_size)

In [ ]:
encoder_embedding_output=encoder_embedding(encoder_input)

In [ ]:
x,state_h,state_c=LSTM(256,return_state=True)(encoder_embedding_output)



**return_state =True** actually means:

the output of an LSTM is the **hidden state (h)** which is x in the above case. Now, we need the **cell state (c)** of LSTM to pass
it to the decoder. so we use return_state=True attribute

Here x and state_h are one and the same which are the hidden state of the LSTM at last time step

In [ ]:
state_h.shape

In [ ]:
x.shape

In [ ]:
state_c.shape

In [ ]:
encoder_states = [state_h, state_c]

# Build a decoder

In [ ]:
decoder_input=Input(shape=(None,)) 
#( This means None * None,any number of inputs and each input any number of words)
# This is as good as Input(shape=decoder_max_lenth,)


In [ ]:
decoder_embedding=Embedding(decoder_vocab_size+1,decoder_embeding_size)

In [ ]:
decoder_embedding_output=decoder_embedding(decoder_input)

In [ ]:
decoder_rnn=LSTM(256,return_sequences=True,return_state=True)

After the embeding layer , now we are supplying 2 inputs to LSTM layer.

1. Previous states ( which would have been set by model to some default values similart to encoder, if we do not pass it to decoder)

2. current embedding output which is the target language in the vector form

In [ ]:
#Initialize initial state with encoder_states
#Output will be all hidden sequences, last 'h' state and last 'c' state

x,_,_=decoder_rnn(decoder_embedding_output,initial_state=encoder_states)

In [ ]:
x.shape

In [ ]:
decoder_dense=Dense(decoder_vocab_size+1,activation='softmax')

In [ ]:
decoder_outputs=decoder_dense(x)

In [ ]:
decoder_outputs.shape

# Now ,the final step

# we have encoder input,decoder input, decoder output and the encoder and decoder layers

In [ ]:
from keras.models import Model

**Here we are sending the decoder inputs during training. This is important point to be noted. 
It is also called Techer Forcing technique. This is to make sure that when the first prediction of the target goes wrong, we dont want to send the predicted output to the next timestep. If we send the predicted targetword to the next step, it is most likely that the entire target sequence will be wrong.
To avoid this, we send the target sequence also in the training**

In [ ]:
model=Model([encoder_input,decoder_input],[decoder_outputs])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

# Train the model

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_onehot,
          batch_size=64,
          epochs=150)
          #validation_split=0.2)

In [ ]:
model.save('seq2seq_eng_hin_training.h5')

# Building the model for prediction 

## Build the encoder model

In [ ]:
encoder_model=Model(encoder_input,encoder_states)

# This means during prediction we are going to send the encoder inputs, i.e. engish text and we get the 
# states ( Cell and hidden) from the encoder

## Build the decoder model

   1. Define both the states h & C initialization
   2. Get the output from Encoder
   3. Define the decoder output
   4. Build the model
   
   





In [ ]:
rnn_units=256 ## LSTM states

In [ ]:
decoder_state_input_h = Input(shape=(rnn_units,)) 
# This means we are going to have 256 hidden states for each sentense becuase the rerutn_state=True for encoder 
# When we say return_state=True, it gives us the state of the LSTM for the last time step. Since we have 
#256 LSTM cells, each cell return one hidden state . 
# Hence for each sequence we will get 256 and for a batch size None*256

In [ ]:
decoder_state_input_c = Input(shape=(rnn_units,))
#Same comment as above for the hidden states. Last time step for each sequence we will get the cell state

In [ ]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# This is a list of both that is going to be the decoder input

In [ ]:
x = decoder_embedding(decoder_input) 
# We are going to use the same embedding layer of the decoder that we have trained.
# But in this case we are not going to send the input to decoder since we dont have any during prediction
# We are just going to send the 'START' word to start predicting the corresponding next work that it understood
# from training for a given combination of cell state and hidden state

In [ ]:
#We will use the layer which we trained earlier i.e. decoder_rnn
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

In [ ]:
#This time we are retaining the decoder states(hidden/cell) for each word
#Why do we need this?
#We need this to predict the sequence of target language words.i.e next word
decoder_states = [state_h, state_c]

In [ ]:
# Last layer is Dense layer . Again we are going to use the same that we have trained before decoder_dense

decoder_outputs = decoder_dense(rnn_outputs)

## Finally, building the decoder model

In [ ]:
decoder_model = Model([decoder_input] + decoder_states_inputs,  #Model inputs i.e. 'START' for the first time and the predicted word next time onwards
                     [decoder_outputs] + decoder_states)

# Predicting Output

In [ ]:
# before starting the prediction we will just have the dictionary of targetword and the index
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [ ]:
len(int_to_word_decoder)

In [ ]:
int_to_word_decoder

In [ ]:
def decode_sentence(input_sequence):
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_sequence)
    
    #Build a sequence with '<start>' - starting sequence for Decoder. As mentioned earlier, we are going 
    # to send only '<Start>' to start predicting the next work for decoder 
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['start']
    
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    
    
    #Initialize predicted sentence
    predicted_sentence = ''
   
    #start the loop
    while not stop_loop:
        
        # We are passsing the initial states as encoder output & 'Start' for the first time.
        # At the end of the loop these values will be updated both target_seq & 
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
       # print('shape of the predicted output',predicted_outputs.shape)
       # print(predicted_outputs)
          #Get the predicted output with highest probability
       # print( 'predicted output',predicted_outputs[0,-1])
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #print(predicted_output)
        
         #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        
        #Check if prediction should stop
        if(predicted_word == 'end' or len(predicted_sentence) > 27):# max_decoder_seq_length=27
            
            stop_loop = True
            continue
            
        
        
         #Updated predicted sentence. First time replace with the prediction of start
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:  
            # second time onwards, concatenate the current prediction with the previous prediction
            
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        
          #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
          
            
            #Update initial states value for decoder. For the first time encoder output is used
            #next time onwards, its the decoder states
        decoder_initial_states_value = [h,c]  
    
    
    
    return predicted_sentence
            
            



# We will call the above prediction with some random text to predict the target sequence

In [ ]:
#Get a random sentence
start_num = np.random.randint(0, high=len(encoder_text) - 10)
#print(start_num)

for i in range(start_num, start_num + 10):
    input_seq = encoder_input_data[i : i+1]
   # print(input_seq)
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )